In [8]:
!pip3 install nltk sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.3 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 10.4 MB/s eta 0:00:00m eta 0:00:01:0101
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 11.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 10.8 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=ec8a026e52141a5f69e8adc17dce46626b4af8ebcd95fa0d6e0cfcb10bcdc6f1
  Stored in directory: /home/alourido/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c913125f8d23c52c1a7c5995e0adb86fef

In [17]:
import os
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import json
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
nltk.download('punkt')
import tqdm



[nltk_data] Downloading package punkt to /home/alourido/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
# Folder path containing the original files
#folder_path = "C:/Users/user/Downloads/dataset_splitted/splits/test"
folder_path = "./data/splits/train"

# Output folder path for JSON files
#output_folder = "C:/Users/user/Downloads/dataset_splitted/splits/test_json"
output_folder = "./data/splits/train_new/"

os.makedirs(output_folder, exist_ok=True)

# Function defined to extract the title and summary from the page.
def extract_title_and_summary(file_path,dirs, limit_link=3,sentences_count=1):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Analyze the HTML content using BeautifulSoup.
    soup = BeautifulSoup(text, 'html.parser')

    # Extract the page title
    title = soup.title.string if soup.title else "No title"

    links = []
    for a in soup.find_all('a'):
        if 'href' in a.attrs:
            if len(links) < limit_link:
                links.append(a['href'])
            else:
                break
        #print("Enlaces en la página:", links)

    # Extract the text from the page's body
    page_text = ' '.join(soup.stripped_strings)

    # Perform summarization using sumy
    parser = PlaintextParser.from_string(page_text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count=sentences_count)  # You can adjust the number of sentences in the summary.
    #print(summary)
    summary_text = ' '.join([str(sentence) for sentence in summary])

    return {"Title": title, "Body": summary_text, "link": links ,"ground_truth": dirs}

# Process all files in the folder
for dirs in os.listdir(folder_path):
    for filename in tqdm.tqdm(os.listdir(os.path.join(folder_path, dirs)), desc="Extrayendo datos"):
        file_path = os.path.join(folder_path, dirs, filename)
        #print(file_path)
        if os.path.isfile(file_path):
            formatted_data = extract_title_and_summary(file_path,dirs)
            #print(formatted_data)
            #print("Processed:", filename)
            #print("Title:", formatted_data["Title"])
            #print("Summary:", formatted_data["Body"])
            #print("Ground Truth:", formatted_data["ground_truth"])
            # Save the data to a JSON fileç
            output_folder_dir=os.path.join(output_folder,dirs)
            os.makedirs(output_folder_dir, exist_ok=True)
            output_json_file = os.path.join(output_folder_dir, os.path.splitext(filename)[0] + ".json")
            with open(output_json_file, 'w', encoding='utf-8') as json_file:
                json.dump(formatted_data, json_file, ensure_ascii=False, indent=4)


Extrayendo datos:  10%|██▏                   | 307/3011 [00:11<01:04, 42.13it/s]/tmp/ipykernel_7618/1659780632.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
Extrayendo datos: 100%|███████████████████████| 744/744 [00:09<00:00, 79.95it/s]


In [21]:
# Folder path containing the original files
#folder_path = "C:/Users/user/Downloads/dataset_splitted/splits/test"
folder_path = "./data/splits/test"

# Output folder path for JSON files
#output_folder = "C:/Users/user/Downloads/dataset_splitted/splits/test_json"
output_folder = "./data/splits/test_new/test"
# Process all files in the folder
for filename in tqdm.tqdm(os.listdir(folder_path), desc="Extrayendo datos"):
        file_path = os.path.join(folder_path, filename)
        #print(file_path)
        if os.path.isfile(file_path):
            formatted_data = extract_title_and_summary(file_path,'unknow')
            #print(formatted_data)
            #print("Processed:", filename)
            #print("Title:", formatted_data["Title"])
            #print("Summary:", formatted_data["Body"])
            #print("Ground Truth:", formatted_data["ground_truth"])
            # Save the data to a JSON fileç
            output_folder_dir=os.path.join(output_folder)
            os.makedirs(output_folder_dir, exist_ok=True)
            output_json_file = os.path.join(output_folder_dir, os.path.splitext(filename)[0] + ".json")
            with open(output_json_file, 'w', encoding='utf-8') as json_file:
                json.dump(formatted_data, json_file, ensure_ascii=False, indent=4)

Extrayendo datos: 100%|█████████████████████| 1659/1659 [01:16<00:00, 21.78it/s]


In [7]:
import json

# Palabras clave a buscar en el campo "Resultado"
palabras_clave = ['course', 'department', 'faculty', 'other', 'project', 'staff', 'student']

# Cargar el JSON desde un archivo
with open('resultados_2023-11-06_03-09-05.json', 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

# Limpiar el campo "Resultado"
for elemento in datos:
    # Encontrar la primera palabra clave en "Resultado"
    resultado = next((palabra for palabra in palabras_clave if palabra in elemento["Resultado"]), "NONE")
    elemento["Resultado"] = resultado

# Escribir el JSON limpio a un nuevo archivo
with open('datos_limpios.json', 'w', encoding='utf-8') as archivo:
    json.dump(datos, archivo, ensure_ascii=False, indent=4)


In [8]:
import json

# Palabras clave a buscar en el campo "Resultado"
palabras_clave = ['course', 'department', 'faculty', 'other', 'project', 'staff', 'student']

# Cargar el JSON desde un archivo
with open('datos_limpios.json', 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

# Lista para guardar los títulos con resultado "NONE"
titulos_con_none = []

# Limpiar el campo "Resultado" y recoger los títulos requeridos
for elemento in datos:
    # Encontrar la primera palabra clave en "Resultado"
    resultado = next((palabra for palabra in palabras_clave if palabra in elemento["Resultado"]), "NONE")
    # Si el resultado es "NONE", añadir el título a la lista
    if resultado == "NONE":
        titulos_con_none.append(elemento["Titulo"])

# Opcional: Escribir los títulos con "NONE" a un archivo de texto
with open('titulos_con_none.txt', 'w', encoding='utf-8') as archivo:
    for titulo in titulos_con_none:
        archivo.write(titulo + '\n')

# Imprimir la lista de títulos
print(titulos_con_none)

['aagelci.json', 'abdjgiz.json', 'acbplmv.json', 'aceolgzx.json', 'ahomx.json', 'aishkqu.json', 'altuxox.json', 'amowrsh.json', 'apakwv.json', 'apmwy.json', 'aqhzllis.json', 'arwvjftm.json', 'asmucefp.json', 'audtaxh.json', 'awlal.json', 'axphpch.json', 'ayteap.json', 'azndzp.json', 'bdwiu.json', 'beoczxhp.json', 'bghlh.json', 'bgxblaq.json', 'bhlwohi.json', 'biaio.json', 'bjwiuevz.json', 'bjzfnftw.json', 'bmmzflr.json', 'bodmvxdf.json', 'bpblgwar.json', 'bphhvc.json', 'bpjzrlxy.json', 'bqbvlmis.json', 'brmtqcj.json', 'brorn.json', 'brwup.json', 'brzpgboe.json', 'bsjgc.json', 'bslps.json', 'btrlegkg.json', 'bttjq.json', 'bvabh.json', 'bvnniatg.json', 'bzavavce.json', 'caaelf.json', 'caeck.json', 'caogdi.json', 'cbuuznl.json', 'ccbje.json', 'cdhwa.json', 'cdruyo.json', 'cehogf.json', 'cfsvmoen.json', 'cftid.json', 'cfwzqt.json', 'chwusgl.json', 'cifmtm.json', 'cifyk.json', 'cioumfyi.json', 'ciuipt.json', 'ciwmb.json', 'cketrvj.json', 'cnhjk.json', 'cnlce.json', 'cqksepwt.json', 'cqqqjx.